In [8]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

We are going to use pyteseract to extract min and max temps so we have the scale range.

In [ ]:
def run_ocr_on_coords(image_path):
    # Hardcoded coordinates for min/max temp regions
    coords = {
        "min_top_left": {"x": 36, "y": 31, "width": 34, "height": 15},
        "max_top_left": {"x": 35, "y": 45, "width": 35, "height": 17},
        "max_right": {"x": 197, "y": 38, "width": 38, "height": 18},
        "min_right": {"x": 199, "y": 261, "width": 36, "height": 19}
    }

    # Load the image
    img = Image.open(image_path)
    results = {}

    for label, coord in coords.items():
        x, y, w, h = coord["x"], coord["y"], coord["width"], coord["height"]
        cropped = img.crop((x, y, x + w, y + h))

        # Preprocessing: grayscale, contrast, sharpen
        gray = ImageOps.grayscale(cropped)
        enhanced = ImageEnhance.Contrast(gray).enhance(2.0)
        sharp = enhanced.filter(ImageFilter.SHARPEN)

        # Run OCR
        text = pytesseract.image_to_string(
            sharp,
            config='--psm 6 -c tessedit_char_whitelist=0123456789.'
        ).strip()

        results[label] = text

    return results

# Example usage
if __name__ == "__main__":
    image_path = "thermal_imges_for_exp\CH01.jpeg"  # Replace with your path
    results = run_ocr_on_coords(image_path)
    for key, val in results.items():
        print(f"{key}: {val}")


min_top_left: 49.7
max_top_left: 71.8
max_right: 71.4
min_right: 49.9
